<a href="https://www.kaggle.com/code/natsu18/optuna-weights-pgs4e3?scriptVersionId=162160031" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
pip install cmaes

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Set the default color palette to "pastel"
sns.set_palette("muted")
import random
import os
from copy import deepcopy
from functools import partial
from itertools import combinations
import random
import gc
import cmaes

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder, CatBoostEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, NMF
from umap import UMAP
from sklearn.manifold import TSNE

# Import libraries for Hypertuning
import optuna

# Import libraries for gradient boosting
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier, Pool

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

2024-02-08 10:28:48.071219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 10:28:48.071383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 10:28:48.281716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
filepath = '/kaggle/input/playground-series-s4e2/'
df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
original = pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')

original = original.reset_index()
original['id'] = original.index + df_test.index[-1] +1
original = original.drop(columns=['index']).set_index('id')

target_col = 'NObeyesdad'

In [4]:
cat_cols = df_test.columns[df_test.nunique()<=7]
num_cols = df_test.columns[df_test.nunique()>7]
cat_cols

Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS'],
      dtype='object')

In [5]:
df_train_copy = df_train.copy(deep=True)
ce_oe = OrdinalEncoder()
df_train_copy[cat_cols] = ce_oe.fit_transform(df_train_copy[cat_cols])

le = LabelEncoder()
df_train_copy[target_col] = le.fit_transform(df_train_copy[target_col])

df_train_copy.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1,1,2.763573,1,0.000000,0.976473,1,1,6
1,2,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2,1,2.000000,1,1.000000,1.000000,2,2,1
2,2,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1,1,1.910378,1,0.866045,1.673584,2,1,0
3,2,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1,1,1.674061,1,1.467863,0.780199,1,1,4
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1,1,1.979848,1,1.967973,0.931721,1,1,6


In [6]:
df_test_copy = df_test.copy(deep=True)
ce_oe = OrdinalEncoder()
df_test_copy[cat_cols] = ce_oe.fit_transform(df_test_copy[cat_cols])

df_test_copy.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
id,,,,,,,,,,,,,,,,
20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,1,2.825629,1,0.855400,0.000000,1,1
20759,2,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,1,3.000000,1,1.000000,0.000000,1,1
20760,2,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,1,2.621877,1,0.000000,0.250502,1,1
20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,1,2.786417,1,0.094851,0.000000,1,1
20762,2,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,1,2.653531,1,0.000000,0.741069,1,1


In [7]:
X_train = df_train_copy.drop([f'{target_col}'], axis=1).reset_index(drop=True)
y_train = df_train_copy[f'{target_col}'].reset_index(drop=True)
X_test = df_test_copy.reset_index(drop=True)

print(f"X_train shape :{X_train.shape} , y_train shape :{y_train.shape}")
print(f"X_test shape :{X_test.shape}")

X_train shape :(20758, 16) , y_train shape :(20758,)
X_test shape :(13840, 16)


In [8]:
class Splitter:
    def __init__(self, test_size=0.2, kfold=True, n_splits=5):
        self.test_size = test_size
        self.kfold = kfold
        self.n_splits = n_splits
        
    def split_data(self, X, y, random_state_list):
        if kfold:
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_idx, val_idx in kf.split(X, y):
                    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                    yield X_train, X_val, y_train, y_val
        else:
            for random_state in random_state_list:
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.test_size, random_state=random_state)
                yield X_train, X_val, y_train, y_val


kfold = True
n_splits = 10
random_state = 4228
random_state_list = [8828] # used by split_data
n_estimators = 9999 # 9999
early_stopping_rounds = 100
verbose = False
device = 'cpu'

splitter = Splitter(kfold=kfold, n_splits=n_splits)        

In [9]:
def objective_lgb(trial):
    """
    Objective function to be maximized.
    """
    scores = []
    lgb_params = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 7,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    
    models = {
            #'xgb': xgb.XGBClassifier(**xgb_params),
            'lgb': lgb.LGBMClassifier(**lgb_params),
            #'cat': CatBoostClassifier(**cb_params),
        }
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, 
                                                                           y_train, 
                                                                           random_state_list = random_state_list)):
        n = i % n_splits
        m = i // n_splits

        for name, model in models.items():
            if name in ['lgb', 'cat']:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], verbose=verbose)

            y_val_pred = model.predict(X_val)
            test_pred = model.predict(X_test)

            score = accuracy_score(y_val, y_val_pred) #(sp.csr_matrix(y_val).toarray() ,sp.csr_matrix(y_val_pred).toarray())
            scores.append(score)

            print(f'Base MODEL {name} [FOLD-{n} SEED-{random_state_list[m]}] accuracy score: {score:.5f}')

    return np.average(scores)

In [10]:
#sampler = optuna.samplers.CmaEsSampler(seed=random_state)
#study_lgb = optuna.create_study(study_name="HyperParams", direction="maximize", sampler=sampler)
#study_lgb.optimize(objective_lgb, n_trials=100)

In [11]:
#print('Best params:', study_lgb.best_params)

In [12]:
def objective_xgb(trial):
    """
    Objective function to be maximized.
    """
    scores = []
    
    xgb_params = {
            'objective'             : 'multi:softprob',
            'num_classes'           : 7,
            'eval_metric'           : "mlogloss",
            'random_state'          : random_state,
            'colsample_bytree'      : trial.suggest_float("colsample_bytree", 1e-8, 1),
            'learning_rate'         : trial.suggest_float("learning_rate", 1e-8, 1),
            'gamma'                 : trial.suggest_float("gamma", 1e-8, 1),
            'max_depth'             : trial.suggest_int("bagging_freq", 1, 10),
            'n_estimators'          : 1100,                         
            'reg_alpha'             : trial.suggest_float("reg_alpha", 1e-8, 1),
            'reg_lambda'            : trial.suggest_float("reg_lambda", 1e-8, 1),
            'min_child_weight'      : trial.suggest_int("min_child_weight", 1, 15),
            'early_stopping_rounds' : 200,
            'verbosity'             : 0,
        }
    
    
    models = {
            'xgb': xgb.XGBClassifier(**xgb_params),
            #'lgb': lgb.LGBMClassifier(**lgb_params),
            #'cat': CatBoostClassifier(**cb_params),
        }
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, 
                                                                           y_train, 
                                                                           random_state_list = random_state_list)):
        n = i % n_splits
        m = i // n_splits

        for name, model in models.items():
            if name in ['lgb', 'cat']:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], verbose=verbose)

            y_val_pred = model.predict(X_val)
            test_pred = model.predict(X_test)

            score = accuracy_score(y_val, y_val_pred) #(sp.csr_matrix(y_val).toarray() ,sp.csr_matrix(y_val_pred).toarray())
            scores.append(score)

            print(f'Base MODEL {name} [FOLD-{n} SEED-{random_state_list[m]}] accuracy score: {score:.5f}')

    return np.average(scores)

In [13]:
#sampler = optuna.samplers.CmaEsSampler(seed=random_state)
#study_xgb = optuna.create_study(study_name="HyperParams_xgb", direction="maximize", sampler=sampler)
#study_xgb.optimize(objective_xgb, n_trials=20)

In [14]:
#print('Best params:', study_xgb.best_params)

In [15]:
def objective_cat(trial):
    """
    Objective function to be maximized.
    """
    scores = []
    
    
    cb_params = {
          'objective'           : 'MultiClass',
          'eval_metric'         : "Accuracy",
          'bagging_temperature' : trial.suggest_float("bagging_temperature", 1e-8, 1),
          'colsample_bylevel'   : trial.suggest_float("colsample_bylevel", 1e-8, 1),
          'iterations'          : 1000,
          'learning_rate'       : trial.suggest_float("learning_rate", 1e-8, 1),
          'od_wait'             : 12,
          'max_depth'           : 7,
          'l2_leaf_reg'         : trial.suggest_float("l2_leaf_reg", 1e-8, 1),
          'min_data_in_leaf'    : 15,
          'random_strength'     : 0.1, 
          'max_bin'             : 100,
          'verbose'             : 0,
          'use_best_model'      : True,
        }
    
    models = {
            #'xgb': xgb.XGBClassifier(**xgb_params),
            #'lgb': lgb.LGBMClassifier(**lgb_params),
            'cat': CatBoostClassifier(**cb_params),
        }
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, 
                                                                           y_train, 
                                                                           random_state_list = random_state_list)):
        n = i % n_splits
        m = i // n_splits

        for name, model in models.items():
            if name in ['lgb', 'cat']:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], verbose=verbose)

            y_val_pred = model.predict(X_val)
            test_pred = model.predict(X_test)

            score = accuracy_score(y_val, y_val_pred) #(sp.csr_matrix(y_val).toarray() ,sp.csr_matrix(y_val_pred).toarray())
            scores.append(score)

            print(f'Base MODEL {name} [FOLD-{n} SEED-{random_state_list[m]}] accuracy score: {score:.5f}')

    return np.average(scores)

In [16]:
#sampler = optuna.samplers.CmaEsSampler(seed=random_state)
#study_cb = optuna.create_study(study_name="HyperParams_cat", direction="maximize", sampler=sampler)
#study_cb.optimize(objective_cat, n_trials=100)

In [17]:
#print('Best params:', study_cb.best_params)

In [18]:
lgb_params = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 7,
        "lambda_l1": 2.0807876056438017e-06,
        "lambda_l2": 5.534843238749359e-07,
        "num_leaves": 188,
        "feature_fraction": 0.7004235334383486,
        "bagging_fraction": 0.7313976216112742,
        "bagging_freq": 5,
        "min_child_samples": 46,
    }
    
xgb_params = {
        'objective'             : 'multi:softprob',
        'num_classes'           : 7,
        'eval_metric'           : "mlogloss",
        'random_state'          : random_state,
        'colsample_bytree'      : 0.3756966579377608,
        'learning_rate'         : 0.46962444326235553,
        'gamma'                 : 0.472547772700957,
        'max_depth'             : 4,
        'n_estimators'          : 1100,                         
        'reg_alpha'             : 0.3067382060405643,
        'reg_lambda'            : 0.7226284404634015,
        'min_child_weight'      : 8,
        'early_stopping_rounds' : 200,
        'verbosity'             : 0,
    }

cb_params = {
      'objective'           : 'MultiClass',
      'eval_metric'         : "Accuracy",
      'bagging_temperature' : 0.5536940121900699,
      'colsample_bylevel'   : 0.753466253196363,
      'iterations'          : 1000,
      'learning_rate'       : 0.46962444326235553,
      'od_wait'             : 12,
      'max_depth'           : 7,
      'l2_leaf_reg'         : 0.3622576775025676,
      'min_data_in_leaf'    : 15,
      'random_strength'     : 0.1, 
      'max_bin'             : 100,
      'verbose'             : 0,
      'use_best_model'      : True,
    }